# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

# NLP
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score

# from sklearn.metrics import confusion_matrix
# 
# from sklearn.base import BaseEstimator, TransformerMixin


[nltk_data] Downloading package punkt to /Users/stingl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/stingl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/stingl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///messages.db')
connection = engine.connect()
df = pd.read_sql_table("messages", con=connection)
X = df.iloc[:, 1]
Y = df.iloc[:, 4:] 

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators = 10)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def show_metrics(Y_test, Y_pred):
    """
    Print f1-score, precision and recall for each category
    
    Parameters:
    -----------
    Y_test: True values,
    Y_pred: Predicted values
    """
    for cat in range(0, Y_test.shape[1]):
        cf=classification_report(Y_test.iloc[:, cat], pd.DataFrame(Y_pred).iloc[:, cat], output_dict=True).get('1')
        if cf is not None:
            print(f"Category:{cat: 3d}, f1-score: {cf['f1-score'] :5.3f}, precision: {cf['precision']: 5.3f}, recall: {cf['recall']: 5.3f}")
        else:
            print(f"Caterogy:{cat: 3d} doesn't have value 1")
                  
Y_pred = pipeline.predict(X_test)                  
show_metrics(Y_test, Y_pred)

Category:  0, f1-score: 0.877, precision:  0.846, recall:  0.910
Category:  1, f1-score: 0.576, precision:  0.765, recall:  0.462
Category:  2, f1-score: 0.000, precision:  0.000, recall:  0.000
Category:  3, f1-score: 0.671, precision:  0.752, recall:  0.607
Category:  4, f1-score: 0.139, precision:  0.641, recall:  0.078
Category:  5, f1-score: 0.127, precision:  0.674, recall:  0.070
Category:  6, f1-score: 0.085, precision:  0.524, recall:  0.046
Category:  7, f1-score: 0.000, precision:  0.000, recall:  0.000
Category:  8, f1-score: 0.150, precision:  0.590, recall:  0.086
Caterogy:  9 doesn't have value 1
Category: 10, f1-score: 0.444, precision:  0.864, recall:  0.299
Category: 11, f1-score: 0.650, precision:  0.823, recall:  0.536
Category: 12, f1-score: 0.489, precision:  0.773, recall:  0.357
Category: 13, f1-score: 0.146, precision:  0.714, recall:  0.081
Category: 14, f1-score: 0.133, precision:  0.778, recall:  0.073
Category: 15, f1-score: 0.022, precision:  1.000, recall

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [7]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.85      0.91      0.88      5941
           1       0.77      0.46      0.58      1333
           2       0.00      0.00      0.00        34
           3       0.75      0.61      0.67      3286
           4       0.64      0.08      0.14       644
           5       0.67      0.07      0.13       414
           6       0.52      0.05      0.08       239
           7       0.00      0.00      0.00       156
           8       0.59      0.09      0.15       267
           9       0.00      0.00      0.00         0
          10       0.86      0.30      0.44       512
          11       0.82      0.54      0.65       878
          12       0.77      0.36      0.49       714
          13       0.71      0.08      0.15       123
          14       0.78      0.07      0.13       192
          15       1.00      0.01      0.02        88
          16       0.68      0.06      0.11       259
          17       0.85    

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1a21e386a8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('mcfl',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
               oob_score=False, random_state=Non

In [10]:
parameters = {
#     'vect__binary': [True, False], 
    'mcfl__estimator__max_depth': [20, 50, 100],
    'mcfl__estimator__min_samples_split': [2, 5, 20],
    'mcfl__estimator__n_estimators': [5, 10, 20]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:   44.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:   49.0s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:  1.9min
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:  3.0min
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:  4.0min
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:  4.8min
[Parallel(n_jobs=12)]: Done  67 out of  81 | elapsed:  7.6min remaining:  1.6min
[Parallel(n_jobs=12)]: Done  76 out of  81 | elapsed:  8.6min remaining:   34.0s
[Parallel(n_jobs=12)]: Done  81 out of  81 | elapsed:  8.9min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=12,
       param_grid={'mcfl__estimator__max_depth': [20, 50, 100], 'mcfl__estimator__min_samples_split': [2, 5, 20], 'mcfl__estimator__n_estimators': [5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=10)

In [11]:
cv.best_params_

{'mcfl__estimator__max_depth': 100,
 'mcfl__estimator__min_samples_split': 5,
 'mcfl__estimator__n_estimators': 20}

In [12]:
cv.best_score_

0.6054176066424246

In [13]:
cv.score(X_test, Y_test)

0.5901008156441563

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
Y_pred = cv.predict(X_test)

show_metrics(Y_test, Y_pred)

Category:  0, f1-score: 0.878, precision:  0.796, recall:  0.979
Category:  1, f1-score: 0.551, precision:  0.857, recall:  0.406
Category:  2, f1-score: 0.000, precision:  0.000, recall:  0.000
Category:  3, f1-score: 0.699, precision:  0.775, recall:  0.636
Category:  4, f1-score: 0.119, precision:  0.700, recall:  0.065
Category:  5, f1-score: 0.142, precision:  0.842, recall:  0.077
Category:  6, f1-score: 0.017, precision:  0.667, recall:  0.008
Category:  7, f1-score: 0.000, precision:  0.000, recall:  0.000
Category:  8, f1-score: 0.154, precision:  0.742, recall:  0.086
Caterogy:  9 doesn't have value 1
Category: 10, f1-score: 0.361, precision:  0.913, recall:  0.225
Category: 11, f1-score: 0.388, precision:  0.863, recall:  0.251
Category: 12, f1-score: 0.416, precision:  0.849, recall:  0.276
Category: 13, f1-score: 0.173, precision:  0.750, recall:  0.098
Category: 14, f1-score: 0.070, precision:  0.778, recall:  0.036
Category: 15, f1-score: 0.043, precision:  0.500, recall

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.80      0.98      0.88      5941
           1       0.86      0.41      0.55      1333
           2       0.00      0.00      0.00        34
           3       0.78      0.64      0.70      3286
           4       0.70      0.07      0.12       644
           5       0.84      0.08      0.14       414
           6       0.67      0.01      0.02       239
           7       0.00      0.00      0.00       156
           8       0.74      0.09      0.15       267
           9       0.00      0.00      0.00         0
          10       0.91      0.22      0.36       512
          11       0.86      0.25      0.39       878
          12       0.85      0.28      0.42       714
          13       0.75      0.10      0.17       123
          14       0.78      0.04      0.07       192
          15       0.50      0.02      0.04        88
          16       0.57      0.02      0.03       259
          17       0.86    

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

# Drop columns with only 1 value
"child_alone" and maybe other columns have only one value. So drop them because they may cause problems with some Classificators.

In [27]:
def drop_single_value_columns(data):
    """
    Drop any column which doesn't contain two diffent values
    
    Parameters:
    -----------
    data: DataFrame
    
    Return value:
    -------------
    DataFrame with column(s) removed
    """
    
    data.columns[data.nunique(axis=0) != 2]
    data.drop(columns=data.columns[data.nunique(axis=0) != 2], inplace=True)
    return data

In [25]:
X = df.iloc[:, 1]
Y = drop_single_value_columns(df.iloc[:, 4:]) 


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=SVC()))
])

parameters = {
#     'mcfl__estimator__kernel' : ['linear', 'rbf', 'sigmoid'],
    'mcfl__estimator__kernel' : ['linear'],
#     'mcfl__estimator__degree' : [3, 4, 5, 6],
#     'mcfl__estimator__kernel' : ['poly'],
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   3 out of   3 | elapsed:  8.1min remaining:    0.0s
[Parallel(n_jobs=12)]: Done   3 out of   3 | elapsed:  8.1min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ity=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=12,
       param_grid={'mcfl__estimator__kernel': ['linear']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=10)

In [26]:
Y_pred = cv.predict(X_test)
print(cv.best_params_)
print(cv.best_score_)
print(classification_report(Y_test, Y_pred)) 

{'mcfl__estimator__kernel': 'linear'}
0.6772289922751838
              precision    recall  f1-score   support

           0       0.86      0.93      0.89      5941
           1       0.78      0.58      0.67      1333
           2       0.00      0.00      0.00        34
           3       0.75      0.69      0.72      3286
           4       0.65      0.20      0.31       644
           5       0.75      0.26      0.39       414
           6       0.79      0.13      0.22       239
           7       0.00      0.00      0.00       156
           8       0.68      0.25      0.37       267
           9       0.75      0.66      0.70       512
          10       0.79      0.76      0.78       878
          11       0.80      0.55      0.65       714
          12       0.76      0.45      0.56       123
          13       0.81      0.13      0.22       192
          14       0.57      0.18      0.28        88
          15       0.62      0.19      0.29       259
          16       0.78 

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [ ]:
X = df.iloc[:, 1]
Y = df.iloc[:, 4:].drop(columns='child_alone')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=SVC()))
])

parameters = {
#     'mcfl__estimator__kernel' : ['linear', 'rbf', 'sigmoid'],
    'mcfl__estimator__degree' : [2, 3, 4, 5, 6],
    'mcfl__estimator__kernel' : ['poly'],
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test)
print(cv.best_params_)
print(cv.best_score_)
print(classification_report(Y_test, Y_pred)) 

In [ ]:
pipeline.get_params()

In [ ]:
from sklearn.svm import LinearSVC
X = df.iloc[:, 1]
Y = df.iloc[:, 4:].drop(columns='child_alone')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=LinearSVC()))
])

parameters = {
    'mcfl__estimator__C' : [1],
    'mcfl__estimator__loss' : ['hinge', 'squared_hinge'],
    'mcfl__estimator__max_iter' : [1000, 10000]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)

In [ ]:
Y_pred = cv.predict(X_test)
print(cv.best_params_)
print(cv.best_score_)
print(classification_report(Y_test, Y_pred))

In [ ]:
cv.get_params()

In [23]:
from sklearn.naive_bayes import MultinomialNB
X = df.iloc[:, 1]
Y = df.iloc[:, 4:] #.drop(columns='child_alone')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=MultinomialNB()))
])

parameters = {
    'mcfl__estimator__alpha' : [0.01, 0.1, 1, 10]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:   30.2s
[Parallel(n_jobs=12)]: Done   3 out of  12 | elapsed:   30.5s remaining:  1.5min
[Parallel(n_jobs=12)]: Done   5 out of  12 | elapsed:   30.6s remaining:   42.8s
[Parallel(n_jobs=12)]: Done   7 out of  12 | elapsed:   30.7s remaining:   21.9s
[Parallel(n_jobs=12)]: Done   9 out of  12 | elapsed:   30.8s remaining:   10.3s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:   33.5s finished
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...fier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=12,
       param_grid={'mcfl__estimator__alpha': [0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=10)

In [24]:
Y_pred = cv.predict(X_test)
print(cv.best_params_)
print(cv.best_score_)
print(classification_report(Y_test, Y_pred))

{'mcfl__estimator__alpha': 0.01}
0.587659155515815
              precision    recall  f1-score   support

           0       0.83      0.94      0.88      5941
           1       0.67      0.59      0.63      1333
           2       0.00      0.00      0.00        34
           3       0.67      0.66      0.66      3286
           4       0.54      0.21      0.30       644
           5       0.49      0.22      0.30       414
           6       0.58      0.06      0.11       239
           7       0.17      0.01      0.02       156
           8       0.55      0.30      0.39       267
           9       0.00      0.00      0.00         0
          10       0.51      0.20      0.28       512
          11       0.61      0.37      0.46       878
          12       0.59      0.27      0.37       714
          13       0.48      0.24      0.32       123
          14       0.51      0.10      0.17       192
          15       0.25      0.02      0.04        88
          16       0.23      0

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being se

In [ ]:
cv.get_params()

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.